In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

from dotenv import load_dotenv

warnings.filterwarnings("ignore")

In [3]:
from app.utils.data_loader import DataLoader
from app.utils.health_score import HealthScore
from app.utils.portfolios import Portfolios
from app.utils.portfolios_repo import PortfoliosRepository
from app.utils.portfolios_service import PortfolioService
from app.utils.portprop_matrices import PortpropMatrices
from app.utils.portprop_matrices_repo import PortpropMatricesRepository
from app.utils.rebalancer import Rebalancer
from app.utils.rebalancer_repo import RebalancerRepository

## Load Data

In [5]:
load_dotenv()  # Load environment variables from .env file
print(os.getenv("LOAD_DATA_FROM_DWH"))

False


In [6]:
data_loader_dwh = DataLoader(load_from_db=True)
data_loader_parquet = DataLoader(load_from_db=False)

In [7]:
data_loader = DataLoader(load_from_db=False)
ppm_repo = PortpropMatricesRepository(data_loader=data_loader)
ports_repo = PortfoliosRepository(data_loader=data_loader)
rebalancer_repo = RebalancerRepository(data_loader=data_loader)

In [9]:
client_out_enriched = ports_repo.load_client_out_product_enriched(
    as_of_date="2025-09-30"
)

In [10]:
client_styles = ports_repo.load_client_style(as_of_date="2025-09-30")

query


In [7]:
client_styles.query("sales_id == '84'").shape

(65053, 14)

In [8]:
client_styles["business_unit"].value_counts()

business_unit
EDGE    82506
PWM      9029
Name: count, dtype: Int64

In [9]:
client_styles.query("customer_id == 39983")

,as_of_date,customer_id,client_full_name_th,client_first_name_en,client_last_name_en,port_investment_style,client_tier,business_unit,client_segment_by_inv_aum,client_sub_segment_by_inv_aum,sales_id,ui_client,sales_first_name_en,sales_team
21855,2025-09-30,39983,วรพล ตันทวีวงศ์,WORAPHON,TONTAWEEWONG,High Risk,Edge,EDGE,Qualified,<2mb,84,N,Digital Edge,Digital Edge


In [12]:
ports_ref_table = {
    "product_mapping": ports_repo.load_product_mapping(as_of_date="2025-09-30"),
    "product_underlying": ports_repo.load_product_underlying(),
}

In [13]:
ppm_ref_dict = {
    "portprop_factsheet": ppm_repo.load_portprop_factsheet(),
    "portprop_benchmark": ppm_repo.load_portprop_benchmark(),
    "portprop_ge_mapping": ppm_repo.load_portprop_ge_mapping(),
    "portprop_fallback": ppm_repo.load_portprop_fallback(),
    "portprop_ret_eow": ppm_repo.load_portprop_ret_eow(),
    "advisory_health_score": ppm_repo.load_advisory_health_score(),
}

In [14]:
rb_ref_dict = {
    "es_sell_list": rebalancer_repo.load_es_sell_list(),
    "product_recommendation_rank_raw": rebalancer_repo.load_product_recommendation_rank_raw(),
    "mandate_allocation": rebalancer_repo.load_mandate_candidates(),
}

## Instances

In [15]:
## Portsfolios
ports_all = Portfolios()
ports_all.set_ref_tables(ports_ref_table)
df_out, df_style, port_ids, port_id_mapping = ports_all.create_portfolio_id(
    client_out_enriched, client_styles, column_mapping=["as_of_date", "customer_id"]
)
ports_all.set_portfolio(df_out, df_style, port_ids, port_id_mapping)

## Portfolio Service
port_service = PortfolioService(ports_all)

## Portprop Matrices
ppm = PortpropMatrices(ppm_ref_dict)

## Health Score
hs = HealthScore()

## Rebalancer
rb = Rebalancer(
    client_investment_style="Moderate High Risk",
    client_classification="UI",
    discretionary_acceptance=0.2,
    new_money=1_000_000,
    product_whitelist=["KKP", "PTTEP"],
    product_blacklist=["KKP GNP", "K-GSELECTU-A(A)"],
)
rb.set_ref_tables(rb_ref_dict)

In [19]:
a = list(port_service.portfolios.df_style['port_investment_style'].unique())

In [22]:
a.remove('Moderate Low Risk')

In [ ]:
len(a)

['Conservative',
 'Bulletproof',
 'High Risk',
 'Unwavering',
 'Aggressive Growth',
 'Moderate High Risk']

## Portfolio (Service)

In [13]:
## get list of all customer ids
print(port_service.get_all_customer_ids()[0:5])

[26527, 25914, 31882, 110343, 24191]


In [15]:
## get single port from customer id
port = port_service.get_client_portfolio(customer_id=39597)

In [16]:
port.df_style.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 45879 to 45879
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   port_id                        1 non-null      int64 
 1   client_full_name_th            1 non-null      string
 2   client_first_name_en           1 non-null      string
 3   client_last_name_en            1 non-null      string
 4   port_investment_style          1 non-null      string
 5   client_tier                    1 non-null      string
 6   business_unit                  1 non-null      string
 7   client_segment_by_inv_aum      1 non-null      string
 8   client_sub_segment_by_inv_aum  1 non-null      string
 9   sales_id                       1 non-null      string
 10  ui_client                      1 non-null      string
 11  sales_first_name_en            1 non-null      string
 12  sales_team                     1 non-null      string
 13  portpo

In [ ]:
## get port outstanding
port.df_out

,customer_id,as_of_date,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,...,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,value,port_id,asset_class_code,weight
195361,14055,2025-09-30,S00088617,RATCH,TRADE,L,THB,RATCH,Listed Securities,Local Equity,...,False,0.0420,False,<NA>,Not Top-Pick,<NA>,1.145489e+07,1866,AA_LE,0.014919
195364,14055,2025-09-30,S00089043,SIRI (in lend),SIDEB,L,THB,SIRI,Listed Securities,Local Equity,...,True,0.0420,False,<NA>,Not Top-Pick,<NA>,1.799901e+06,1866,AA_LE,0.002344
197967,14055,2025-09-30,S00086880,LH (in lend),SIDEB,L,THB,LH,Listed Securities,Local Equity,...,False,0.0420,False,Switch to SPALI,Not Top-Pick,<NA>,1.006560e+05,1866,AA_LE,0.000131
200606,14055,2025-09-30,S00086297,OR,TRADE,L,THB,OR,Listed Securities,Local Equity,...,False,0.0420,False,Switch to TOP/SPRC,Not Top-Pick,<NA>,2.450920e+04,1866,AA_LE,0.000032
203098,14055,2025-09-30,S00080142,TLI,TRADE,L,THB,TLI,Listed Securities,Local Equity,...,False,0.0420,False,<NA>,Not Top-Pick,<NA>,4.240000e+05,1866,AA_LE,0.000552
203112,14055,2025-09-30,S00082395,CPNREIT,TRADE,L,THB,CPNREIT,Listed Securities,Alternative,...,True,0.0420,False,<NA>,Not Top-Pick,<NA>,7.772000e+06,1866,AA_ALT,0.010122
203125,14055,2025-09-30,S00087551,BTSGIF,TRADE,L,THB,BTSGIF,Listed Securities,Alternative,...,False,0.0420,False,<NA>,Not Top-Pick,<NA>,1.037400e+05,1866,AA_ALT,0.000135
203131,14055,2025-09-30,S00087906,SCB,TRADE,L,THB,SCB,Listed Securities,Local Equity,...,True,0.0420,False,<NA>,Not Top-Pick,<NA>,5.156705e+07,1866,AA_LE,0.067160
203142,14055,2025-09-30,S00249304,AXTRART,TRADE,L,THB,AXTRART,Listed Securities,Alternative,...,False,0.0420,False,<NA>,Not Top-Pick,<NA>,1.328400e+07,1866,AA_ALT,0.017301
204998,14055,2025-09-30,C00004343,Equity Collateral,TRADE,L,THB,Equity Collateral,Cash,Cash and Cash Equivalent,...,True,0.0480,False,<NA>,Not Top-Pick,<NA>,1.232468e+07,1866,AA_CASH,0.016052


In [17]:
ppm.df_port_fs

,symbol,bm_name,asset_class,weight
0,KKPEQTHB,TMBMONEY,Cash and Cash Equivalent,0.01
1,KKPFITHB,SETPREIT50% + FSTREI50%,Alternative,0.00
2,KKPFITHB,BB Barclay Global Aggregate Hedged-USD (70%) ...,Fixed Income,0.00
3,KKPFITHB,Bloomberg Commodity TR,Alternative,0.00
4,KKPFITHB,Dow Jones Brookfield Global Infrastructure Ind...,Alternative,0.00
...,...,...,...,...
1153,KKPEQTHB,SET TRI,Local Equity,0.00
1154,KKPEQTHB,SETPF&REIT,Alternative,0.00
1155,KKPEQTHB,S&P Global REIT Index USD (TR),Alternative,0.00
1156,KKPEQTHB,"All Thai Gov Bond, Avg. Dur. 7.79 (25/09/23), ...",Fixed Income,0.00


In [18]:
# get port allocation lookthrough
port.get_portfolio_asset_allocation_lookthrough(ppm)

asset_class,port_id,aa_alt,aa_cash,aa_fi,aa_ge,aa_le
0,19384,0.011331,0.003492,0.006101,0.962515,0.01656


In [19]:
# get model allocation
port.get_model_asset_allocation_lookthrough(ppm)

,port_id,port_investment_style,portpop_styles,aa_alt_model,aa_cash_model,aa_fi_model,aa_ge_model,aa_le_model
0,19384,Unwavering,Aggressive,0.105,0.055,0.08,0.684,0.076


In [20]:
port.df_style

,port_id,client_full_name_th,client_first_name_en,client_last_name_en,port_investment_style,client_tier,business_unit,client_segment_by_inv_aum,client_sub_segment_by_inv_aum,sales_id,ui_client,sales_first_name_en,sales_team,portpop_styles
45879,19384,ธันย***************,THAN******,PHIN*******,Unwavering,Edge,EDGE,Qualified,<2mb,84,N,Digi********,Digi********,Aggressive


## Healthscore

In [21]:
## get client health score
health_score, health_score_comp = port.get_portfolio_health_score(ppm, hs)

In [22]:
health_score

,port_id,expected_return,expected_return_model,score_ret,volatility,volatility_model,score_vol,score_portfolio_risk,acd,score_acd,ged,score_ged,score_diversification,score_bulk_risk,score_issuer_risk,score_non_cover_global_stock,score_non_cover_local_stock,score_non_cover_mutual_fund,score_not_monitored_product,health_score
0,19384,0.085251,0.073815,0,0.165607,0.1364,-1,-1,0.313036,0,0.242219,-1,-1,-2,0,0,0,0,0.0,6.0


In [23]:
health_score_comp

,port_id,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,value,...,ge_other,expected_return,volatility,is_bulk_risk,underlying_company,issure_risk_group,coverage_prdtype,score_non_cover_global_stock,score_non_cover_local_stock,score_non_cover_mutual_fund
0,19384,M00127344,KKP CorePath Extra-ES,TRADE,L,THB,KKP CorePath Extra-ES,Mutual Fund,Allocation,11848.28,...,0.003510,6.798462e-03,1.115429e-02,False,<NA>,NaN,MUTUAL_FUND,0,0,0
1,19384,M00162119,KKP PGE-UH,TRADE,L,THB,KKP PGE-UH,Mutual Fund,Global Equity,13105.20,...,0.006811,7.423268e-03,1.627748e-02,False,<NA>,NaN,MUTUAL_FUND,0,0,0
2,19384,M00228153,K-GPINUH-A(A),TRADE,L,THB,K-GPINUH-A(A),Mutual Fund,Global Equity,48481.68,...,0.025196,3.102828e-02,6.021727e-02,True,<NA>,NaN,MUTUAL_FUND,0,0,0
3,19384,C00142044,SS_2013096061,TRADE,L,THB,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,0.83,...,NaN,2.442299e-07,-2.927335e-10,False,<NA>,NaN,N/A,0,0,0
4,19384,M00248078,KKP EWUS500-UH,TRADE,L,THB,KKP EWUS500-UH,Mutual Fund,Global Equity,52011.99,...,0.000000,3.328768e-02,6.492894e-02,True,<NA>,NaN,MUTUAL_FUND,0,0,0
5,19384,M00072598,KFGBRAND-A,TRADE,L,THB,KFGBRAND-A,Mutual Fund,Global Equity,10489.50,...,0.005452,6.713281e-03,1.302861e-02,False,<NA>,NaN,MUTUAL_FUND,0,0,0


In [ ]:
health_score_comp.columns

Index(['port_id', 'product_id', 'src_sharecodes', 'desk', 'port_type',
       'currency', 'product_display_name', 'product_type_desc',
       'asset_class_name', 'value', 'weight', 'aa_alt', 'aa_cash', 'aa_fi',
       'aa_ge', 'aa_le', 'ge_em', 'ge_eur', 'ge_jp', 'ge_us', 'ge_other',
       'expected_return', 'volatility', 'is_bulk_risk', 'underlying_company',
       'issure_risk_group', 'coverage_prdtype', 'score_non_cover_global_stock',
       'score_non_cover_local_stock', 'score_non_cover_mutual_fund'],
      dtype='object')

## Rebalancer

In [25]:
new_port, recommendations = rb.rebalance(port, ppm, hs)

In [26]:
recommendations[0:3].to_json()

'{"transaction_no":{"0":1,"1":2,"2":3},"batch_no":{"0":1,"1":2,"2":2},"port_id":{"0":19384,"1":19384,"2":19384},"product_id":{"0":"CTHB00000000","1":"M00073326","2":"CTHB00000000"},"src_sharecodes":{"0":"Cash Proxy THB","1":"KKP CorePath Balanced","2":"Cash Proxy THB"},"desk":{"0":"TRADE","1":"TRADE","2":"TRADE"},"port_type":{"0":"L","1":"L","2":"L"},"currency":{"0":"THB","1":"THB","2":"THB"},"product_display_name":{"0":"Cash Proxy THB","1":"KKP CorePath Balanced","2":"Cash Proxy THB"},"product_type_desc":{"0":"Cash","1":"Mutual Fund","2":"Cash"},"asset_class_name":{"0":"Cash and Cash Equivalent","1":"Allocation","2":"Cash and Cash Equivalent"},"value":{"0":null,"1":0.0,"2":null},"weight":{"0":null,"1":0.0,"2":null},"flag":{"0":"new_money","1":"discretionary_buy","2":"cash_proxy_funding"},"expected_weight":{"0":null,"1":0.2,"2":null},"action":{"0":"funding","1":"buy","2":"funding"},"amount":{"0":1000000.0,"1":227187.496,"2":-227187.496},"flag_msg":{"0":"Add new money to the portfolio."

In [ ]:
recommendations["action"].value_counts()

action
funding    19
sell       15
buy         3
Name: count, dtype: int64

In [ ]:
recommendations

,transaction_no,batch_no,port_id,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,value,weight,flag,expected_weight,action,amount,flag_msg
0,1,1,1866,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,new_money,NaN,funding,1.000000e+06,Add new money to the portfolio.
1,2,2,1866,S00086297,OR,TRADE,L,THB,OR,Listed Securities,Local Equity,2.450920e+04,0.000032,not_monitored_product,0.0,sell,-2.450920e+04,Reduce or exit to manage product not actively ...
2,3,2,1866,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,2.450920e+04,Fund transactions through the cash proxy posit...
3,4,3,1866,S00086880,LH (in lend),SIDEB,L,THB,LH,Listed Securities,Local Equity,1.006560e+05,0.000131,not_monitored_product,0.0,sell,-1.006560e+05,Reduce or exit to manage product not actively ...
4,5,3,1866,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,1.006560e+05,Fund transactions through the cash proxy posit...
5,6,4,1866,S00087551,BTSGIF,TRADE,L,THB,BTSGIF,Listed Securities,Alternative,1.037400e+05,0.000135,not_monitored_product,0.0,sell,-1.037400e+05,Reduce or exit to manage product not actively ...
6,7,4,1866,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,1.037400e+05,Fund transactions through the cash proxy posit...
7,8,5,1866,S00085703,SCGP (in lend),SIDEB,L,THB,SCGP,Listed Securities,Local Equity,1.171584e+05,0.000152,not_monitored_product,0.0,sell,-1.171584e+05,Reduce or exit to manage product not actively ...
8,9,5,1866,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,1.171584e+05,Fund transactions through the cash proxy posit...
9,10,6,1866,S00088451,PL,TRADE,L,THB,PL,Listed Securities,Local Equity,2.005898e+05,0.000261,not_monitored_product,0.0,sell,-2.005898e+05,Reduce or exit to manage product not actively ...


In [ ]:
new_port.df_out

,port_id,product_id,src_sharecodes,desk,port_type,currency,value,product_display_name,product_type_desc,asset_class_name,...,is_risky_asset,coverage_prdtype,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,asset_class_code,weight
0,1866,C00004343,Equity Collateral,TRADE,L,THB,1.232468e+07,Equity Collateral,Cash,Cash and Cash Equivalent,...,False,N/A,True,0.0400,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.016031
1,1866,C00020399,GIS THB Collateral,GIS,L,THB,2.118697e+05,GIS THB Collateral,Cash,Cash and Cash Equivalent,...,False,N/A,True,0.0480,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.000276
2,1866,C00051771,SS_2001651092(-2121),TRADE,L,THB,7.840337e+04,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,...,False,N/A,True,0.0400,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.000102
3,1866,C00052573,SS_2001659490(-0436),TRADE,L,THB,4.990886e+05,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,...,False,N/A,True,0.0400,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.000649
4,1866,CTHB00000000,Cash Proxy THB,TRADE,L,THB,3.475256e+08,Cash Proxy THB,Cash,Cash and Cash Equivalent,...,False,N/A,True,0.0400,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.452025
5,1866,DTHB00000000,KKPBATHB,TRADE,L,THB,1.537640e+08,KKPBATHB,Mandate,Allocation,...,False,N/A,True,0.0700,False,<NA>,Not Top-Pick,<NA>,NaN,0.200000
6,1866,M00073313,KKP PLUS,TRADE,L,THB,3.341168e+06,KKP PLUS,Mutual Fund,Cash and Cash Equivalent,...,False,MUTUAL_FUND,True,0.0160,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.004346
7,1866,M00078586,ES-TM,TRADE,L,THB,7.167682e+06,ES-TM,Mutual Fund,Cash and Cash Equivalent,...,False,MUTUAL_FUND,False,0.0115,False,<NA>,Not Top-Pick,<NA>,AA_CASH,0.009323
8,1866,M00162119,KKP PGE-UH,TRADE,L,THB,7.688200e+07,KKP PGE-UH,Mutual Fund,Global Equity,...,True,MUTUAL_FUND,True,0.0770,False,<NA>,Top-Pick,<NA>,AA_GE,0.100000
9,1866,M00228153,K-GPINUH-A(A),TRADE,L,THB,7.688200e+07,K-GPINUH-A(A),Mutual Fund,Global Equity,...,True,MUTUAL_FUND,True,0.0870,False,<NA>,Top-Pick,<NA>,AA_GE,0.100000


In [27]:
health_score, health_score_comp = new_port.get_portfolio_health_score(ppm, hs)

In [28]:
health_score["health_score"].values[0]

np.float64(8.0)